In [49]:
import pandas as pd
df = pd.read_excel('含可转债，中报业绩预增.xlsx')

In [94]:
import pandas as pd
import numpy as np
import pickle
import time 
import os
from datetime import datetime
from operator import itemgetter

data_folder_path = '../'
raw_data_path = "../raw_data"
storage_path = "../database_storage"
daily_path = "../daily"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
DCBS_path = os.path.join(storage_path, "DCBS.pkl")

with open('database_ByZCG\\DCBS_DataFrame.pkl', 'rb') as f:  
    DCBS_DataFrame = pickle.load(f)
    
with open('database_ByZCG\\A_ConvertibleBondTimeSeries_DataFrame.pkl', 'rb') as f:  
    A_ConvertibleBondTimeSeries_DataFrame = pickle.load(f)
    
# 只提取A区中存活的转债的时间序列信息
A_DF = A_ConvertibleBondTimeSeries_DataFrame[A_ConvertibleBondTimeSeries_DataFrame['ia']==1].copy()

print(df)
print(A_DF)

         股票代码   股票简称   现价(元)  涨跌幅(%) 未清偿可转债简称\n2023.07.14 业绩预告类型\n2023.06.30  \
0   603569.SH   长久物流   10.65  -1.480                 长久转债               大幅上升   
1   300779.SZ   惠城环保   50.73  -1.591                 惠城转债                 扭亏   
2   002616.SZ   长青集团    6.31  -3.664                 长集转债               大幅上升   
3   002772.SZ   众兴菌业   10.07  -0.592                 众兴转债                 扭亏   
4   002969.SZ   嘉美包装    4.00   0.251                 嘉美转债                 扭亏   
5   603499.SH   翔港科技    9.51  -0.523                 翔港转债               大幅上升   
6   688223.SH   晶科能源   12.50  -3.772                 晶能转债               大幅上升   
7   603518.SH   锦泓集团   10.50  -6.915                 维格转债               大幅上升   
8   600903.SH   贵州燃气    8.63  -0.690                 贵燃转债                 扭亏   
9   603466.SH    风语筑   13.97   0.143                 风语转债                 扭亏   
10  300645.SZ   正元智慧   28.35   4.151                正元转02               大幅上升   
11  603679.SH   华体科技   14.57   1.040    

In [95]:
today_DF = A_ConvertibleBondTimeSeries_DataFrame[A_ConvertibleBondTimeSeries_DataFrame['date'] == '2023-07-18'].copy()

def bond_code_to_pure_number(x):
    return x[0:6]

today_DF['pure_number_bond_code'] = today_DF['bond_code'].apply(lambda x : int(x[0:6]))
print(today_DF)

output_df = pd.merge(df, today_DF[['pure_number_bond_code','qs','qs15','qs30','xx','cpr','dp','dl','ytm','bl','trt','yl']],left_on='未清偿可转债代码\n2023.07.14', right_on='pure_number_bond_code')
del output_df['pure_number_bond_code']

output_df.rename(columns={'qs' : '强赎',
                          'qs15' : '强赎15',
                          'qs30' : '强赎30',
                          'xx' : '下修',
                          'dp' : '价格（收盘价全价）',
                          'cpr' : '转股溢价率',
                          'dl' : '双低',
                          'ytm': 'ytm',
                          'bl' : '债券余额（亿）',
                          'trt' : '换手率',
                          'yl' : '剩余期限（年）'},inplace=True)


output_df.to_excel('经过处理后的结果.xlsx',sheet_name='结果',index=False)

                      bond_code        date    cpr       dp      bl     trt  \
id                                                                            
110088.SH,2023-07-18  110088.SH  2023-07-18  35.59  118.308  30.000    0.82   
113545.SH,2023-07-18  113545.SH  2023-07-18  39.02  116.410   9.865    1.03   
113631.SH,2023-07-18  113631.SH  2023-07-18   7.27  127.701   9.299    4.39   
123128.SZ,2023-07-18  123128.SZ  2023-07-18  65.14  102.568  13.790    1.86   
123156.SZ,2023-07-18  123156.SZ  2023-07-18  12.52  129.998   3.970    9.11   
...                         ...         ...    ...      ...     ...     ...   
127086.SZ,2023-07-18  127086.SZ  2023-07-18  26.80  131.000  31.600    5.19   
111014.SH,2023-07-18  111014.SH  2023-07-18  43.18  131.559   6.000   16.03   
123200.SZ,2023-07-18  123200.SZ  2023-07-18  82.60  188.760   3.960   87.60   
127087.SZ,2023-07-18  127087.SZ  2023-07-18  30.04  139.395   4.629  645.20   
123201.SZ,2023-07-18  123201.SZ  2023-07-18  32.03  